In [1]:
import pandas as pd
import numpy as np
import os
import re

### 1 - Load and concatenate all the CSV files
The empty columns will be dropped

In [2]:
# Load all CSV files from `../data/interim` into a list of DataFrames
# Add a new column to each DataFrame with the name of the file
csv_path = os.path.join('..', 'data', 'interim')
dfs = []

for f in sorted(os.listdir(csv_path)):
    if f.endswith('.csv'):
        df = pd.read_csv(os.path.join(csv_path, f))
        df['file'] = f
        dfs.append(df)

In [3]:
# Check the dimension of each DataFrame
dims = [df.shape for df in dfs]

for i in range(len(dims)):
    print(dims[i])


print()

# Check the columns of each DataFrame and check the differences
columns = [df.columns for df in dfs]
diffs = [list(set(columns[0]) - set(col)) for col in columns[1:]]

for i in range(len(diffs)):
    print(diffs[i])

(35105, 408)
(34473, 413)
(33908, 413)
(33551, 404)
(34718, 429)
(32233, 447)

['BIOTER', 'ULTRASND', 'NOTOBAC', 'MDSP', 'PREGNANT', 'RACE', 'GESTWK', 'ETHNIC']
['BIOTER', 'ULTRASND', 'PAYTYPE', 'NOTOBAC', 'MDSP', 'PREGNANT', 'RACE', 'GESTWK', 'ETHNIC']
['OTHPR12D', 'BIOTER', 'RETPRN', 'RACEFL', 'OTHPROC3', 'SCOPWI1R', 'SCOPPROC', 'OTHPR22D', 'DIAGSC2', 'OTHPR13D', 'MDSP', 'CASTAGE', 'SCOPWI2R', 'DIAGSC1', 'RACE', 'NOFU', 'DIAGSC1R', 'DIGSC23D', 'ULTRASND', 'DIGSC22D', 'PAYTYPE', 'SCOPEWI2', 'HOSPICE', 'OTHPROC2R', 'RADTHER', 'OTHPROC3R', 'OTHPROC4', 'SCOPEWI1', 'OTHPROC1', 'DIGSC13D', 'SPIRO', 'OTHPR33D', 'DIAGSC2R', 'OTHPROC4R', 'OTHPROC2', 'OTHPROC', 'SCPWI22D', 'SCPWI13D', 'BLANK', 'RACEETH', 'ORTHO', 'OTHPR23D', 'DIGSC12D', 'SCPWI12D', 'REFERED', 'GESTWK', 'OTHPROC1R', 'TOTDIAG', 'ETHNIC', 'OTHPR32D', 'ELECTROL', 'SCPWI23D', 'TELEPHON', 'OTHDIAG', 'ADMITHOS', 'DMP', 'OTHPR43D', 'NOTOBAC', 'TOTNONMED', 'NONMED', 'OTHPR42D', 'PREGNANT', 'DIAGSCRN']
['OTHPR12D', 'BIOTER', 'EPUBHTHO',

In [4]:
# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)
print('The shape of the concatenated DataFrame is:')
print(df.shape)
print()

# Drop the columns with all NaN values
print('The shape of the DataFrame after dropping columns with all NaN values is:')
df = df.dropna(axis=1, how='all')
print(df.shape)

The shape of the concatenated DataFrame is:
(203988, 544)

The shape of the DataFrame after dropping columns with all NaN values is:
(203988, 532)


### 2 - Check the distribution of NaN values

In [5]:
# Group by the `file` column and count the number of NaN values in each group
# Drop the columns with zero NaN values
# Transpose the resulting DataFrame and add a sum column
# Sort the DataFrame by the sum column
nan_counts = df.groupby('file').apply(lambda x: x.isna().sum(), include_groups=False)
nan_counts = nan_counts.loc[:, (nan_counts != 0).any(axis=0)]
nan_counts = nan_counts.T
nan_counts['sum'] = nan_counts.sum(axis=1)
nan_counts = nan_counts.sort_values(by='sum', ascending=False)
print(nan_counts)

file      opd2006.csv  opd2007.csv  opd2008.csv  opd2009.csv  opd2010.csv  \
COMSTAT8        33241        31864        31054        30433        31182   
CONTSUB8        33241        31864        31054        30433        31182   
PRESCR8         33241        31864        31054        30433        31182   
COMSTAT7        32465        30844        30050        29361        30000   
CONTSUB7        32465        30844        30050        29361        30000   
...               ...          ...          ...          ...          ...   
PAPUNSP             0            0            0            0            0   
URBANRUR            0            0            0            0            0   
CCS                 0            0            0            0            0   
PAPCONV             0            0            0            0            0   
PASTVIS          1664            0            0            0            0   

file      opd2011.csv     sum  
COMSTAT8        28841  186615  
CONTSUB8   

In [6]:
# Check the percentage of NaN values in each column in each group
nan_perc = df.groupby('file').apply(lambda x: x.isna().mean(), include_groups=False)
nan_perc = nan_perc.loc[:, (nan_perc != 0).any(axis=0)]
nan_perc = nan_perc.T
nan_perc['sum'] = nan_perc.sum(axis=1) / len(dfs)
nan_perc = nan_perc.sort_values(by='sum', ascending=False)
print(nan_perc)

file      opd2006.csv  opd2007.csv  opd2008.csv  opd2009.csv  opd2010.csv  \
COMSTAT8     0.946902     0.924318     0.915831     0.907067     0.898151   
CONTSUB8     0.946902     0.924318     0.915831     0.907067     0.898151   
PRESCR8      0.946902     0.924318     0.915831     0.907067     0.898151   
CONTSUB7     0.924797     0.894729     0.886222     0.875115     0.864105   
COMSTAT7     0.924797     0.894729     0.886222     0.875115     0.864105   
...               ...          ...          ...          ...          ...   
PREGTEST     1.000000     0.000000     0.000000     0.000000     0.000000   
MHP          1.000000     0.000000     0.000000     0.000000     0.000000   
PAPCONV      0.000000     0.000000     0.000000     0.000000     0.000000   
OPDWT        0.993306     0.000000     0.000000     0.000000     0.000000   
PASTVIS      0.047401     0.000000     0.000000     0.000000     0.000000   

file      opd2011.csv       sum  
COMSTAT8     0.894766  0.914506  
CONTSUB

In [7]:
# Check the columns that have all NaN values in some file 
all_nan_cols = nan_perc.drop('sum', axis=1)[(nan_perc == 1).any(axis=1)]

# Count the number of files that have all NaN values in each column
all_nan_cols['counts_of_files'] = all_nan_cols.sum(axis=1)
print(all_nan_cols.head(5))
print()
print('The number of columns that have all NaN values in some file is:')
print(len(all_nan_cols))
print()

# Check the columns that more than 4 files have all NaN values
more_than_4_nan_cols = all_nan_cols[all_nan_cols['counts_of_files'] > 4]
print(more_than_4_nan_cols.head(5))
print()
print('The number of columns that more than 4 files have all NaN values is:')
print(len(more_than_4_nan_cols))
print()

# Check the columns that more than 3 files have all NaN values
more_than_3_nan_cols = all_nan_cols[all_nan_cols['counts_of_files'] == 3]
print(more_than_3_nan_cols.head(5))
print()
print('The number of columns that more than 3 files have all NaN values is:')
print(len(more_than_3_nan_cols))
print()

file       opd2006.csv  opd2007.csv  opd2008.csv  opd2009.csv  opd2010.csv  \
BLANK2             1.0          1.0          1.0          1.0          0.0   
PAP                1.0          1.0          1.0          1.0          1.0   
EBILLANYO          1.0          1.0          1.0          1.0          1.0   
EMEDALGO           1.0          1.0          1.0          1.0          1.0   
ESETSO             1.0          1.0          1.0          1.0          1.0   

file       opd2011.csv  counts_of_files  
BLANK2             1.0              5.0  
PAP                0.0              5.0  
EBILLANYO          0.0              5.0  
EMEDALGO           0.0              5.0  
ESETSO             0.0              5.0  

The number of columns that have all NaN values in some file is:
202

file       opd2006.csv  opd2007.csv  opd2008.csv  opd2009.csv  opd2010.csv  \
BLANK2             1.0          1.0          1.0          1.0          0.0   
PAP                1.0          1.0          1.0     

In [8]:
# Drop the columns that have all NaN values in some file
print('The shape of the DataFrame after dropping columns with all NaN values is:')
print(df.shape)
print()

print('The shape of the DataFrame after dropping columns that have all NaN values in some file is:')
df_wo_allnan = df.drop(all_nan_cols.index, axis=1)
print(df_wo_allnan.shape)

The shape of the DataFrame after dropping columns with all NaN values is:
(203988, 532)

The shape of the DataFrame after dropping columns that have all NaN values in some file is:
(203988, 330)


In [9]:
print(df_wo_allnan.columns.to_list())

['VMONTH', 'VDAYR', 'AGE', 'SEX', 'USETOBAC', 'PAYPRIV', 'PAYMCARE', 'PAYMCAID', 'PAYWKCMP', 'PAYSELF', 'PAYNOCHG', 'PAYOTH', 'PAYDK', 'INJDET', 'INJURY', 'RFV1', 'RFV2', 'RFV3', 'RFV13D', 'RFV23D', 'RFV33D', 'PRIMCARE', 'REFER', 'SENBEFOR', 'PASTVIS', 'MAJOR', 'DIAG1', 'DIAG2', 'DIAG3', 'DIAG13D', 'DIAG23D', 'DIAG33D', 'PRDIAG1', 'PRDIAG2', 'PRDIAG3', 'ARTHRTIS', 'ASTHMA', 'CANCER', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS', 'NOCHRON', 'TOTCHRON', 'HTIN', 'WTLB', 'BMI', 'TEMPF', 'BPSYS', 'BPDIAS', 'BREAST', 'PELVIC', 'RECTAL', 'SKIN', 'DEPRESS', 'ANYIMAGE', 'BONEDENS', 'MAMMO', 'MRI', 'XRAY', 'OTHIMAGE', 'CBC', 'GLUCOSE', 'HGBA', 'CHOLEST', 'PSA', 'OTHERBLD', 'SIGCOLON', 'BIOPSY', 'CHLAMYD', 'HPVDNA', 'EKG', 'URINE', 'HTTAKE', 'WTTAKE', 'TEMPTAKE', 'BLODPRES', 'HEALTHED', 'ASTHMAED', 'DIETNUTR', 'EXERCISE', 'GRWTHDEV', 'INJPREV', 'STRESMGT', 'TOBACED', 'WTREDUC', 'OTHLTHED', 'TOTHLTED', 'CAM', 'DME', 'HOMEHLTH', 'PT', 'SPOCTHER

### 3 - Feature definition and selection
Not including "DRUG-RELATED INFO FOR MEDICATION #1-8" or "DIAGNOSIS RECODE"

In [10]:
# DATE OF VISIT
dateOfVisit = ['VMONTH', 'VYEAR', 'VDAYR', 'YEAR']

# Indepedent variables
demographics = [
    'AGE', 'SEX', 'PREGNANT', 'GESTWEEK', 'ETHNIC', 'RACE', 'USETOBAC', 'NOTOBAC'
]
payment = ['PAYPRIV', 'PAYMCARE', 'PAYMCAID', 'PAYWKCMP', 'PAYSELF', 'PAYNOCHG', 'PAYOTH', 'PAYDK', 'PAYTYPE']
visitReason = [
    'INJDET',
    'INJURY',
    'MAJOR', 'RFV1', 'RFV2', 'RFV3'
]
patientClinicHistory = ['SENBEFOR', 'PASTVIS']

# Supplementary Independent Variables
vitalSigns = ['HTIN', 'WTLB', 'BMI', 'TEMPF', 'BPSYS', 'BPDIAS']
imputedFields = ['BDATEFL', 'SEXFL', 'ETHNICFL', 'RACEFL', 'SENBEFL', 'PASTFL']

# Not sure if Independent or Dependent variables
physicianDiagnoses = ['DIAG1', 'DIAG2', 'DIAG3']
differentialDiagnoses = ['PRDIAG1', 'PRDIAG2', 'PRDIAG3']
presentSymptomsStatus = [
    'ARTHRTIS', 'ASTHMA', 'CANCER', 'CASTAGE', 'CEBVD', 'CHF', 
    'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 
    'HTN', 'IHD', 'OBESITY', 'OSTPRSIS',
    'NOCHRON', 'TOTCHRON',
    'DMP'
]

# Dependent variables
diagnosticScreeningServices = [
    #'DIAGSCRN', 'TOTDIAG',
    'BREAST', 'PELVIC', 'RECTAL', 'SKIN', 'DEPRESS',
    #'ANYIMAGE',
    'BONEDENS', 'MAMMO', 'MRI', 'ULTRASND', 'XRAY', 'OTHIMAGE',
    'CBC', 'ELECTROL', 'GLUCOSE', 'HGBA', 'CHOLEST', 'PSA', 'OTHERBLD',
    #'SCOPPROC', 'SIGCOLON',
    'SCOPEWI1', 'SCOPEWI2',
    'BIOPSY', 'CHLAMYD', 'PAPCONV', 'PAPLIQ', 'PAPUNSP', 'HPVDNA', 'EKG', 'SPIRO', 'URINE',
    'HTTAKE', 'WTTAKE', 'TEMPTAKE', 'BLODPRES',
    #'OTHDIAG',
    'DIAGSC1', 'DIAGSC2',
]
healthEducation = [
    #'HEALTHED', 'TOTHLTED',
    'ASTHMAED', 'DIETNUTR', 'EXERCISE', 'GRWTHDEV', 'INJPREV', 'STRESMGT', 'TOBACED', 'WTREDUC', 'OTHLTHED'
]
nonMedicationTreatments = [
    #'NONMED', 'TOTNONMED',
    'CAM', 'DME', 'HOMEHLTH', 'HOSPICE', 'PT', 'RADTHER', 'SPOCTHER', 'PSYCHOTH', 'OTHMNTL', 'EXCISION', 'ORTHO', 'WOUND',
    #'OTHPROC',
    'OTHPROC1', 'OTHPROC2', 'OTHPROC3', 'OTHPROC4'
]
medicationsAndImmunizations = [
    #'MED', 'NUMMED',
    'MED1', 'MED2', 'MED3', 'MED4', 'MED5', 'MED6', 'MED7', 'MED8',
    'NCMED1', 'NCMED2', 'NCMED3', 'NCMED4', 'NCMED5', 'NCMED6', 'NCMED7', 'NCMED8',
    'NUMNEW', 'NUMCONT'
]
providersSeen = [
    'NOPROVID', 'PHYS', 'PHYSASST', 'NPNMW', 'RNLPN', 'OTHPROV'
]
visitDisposition = [
    'NODISP', 'NOFU', 'RETPRN', 'REFOTHMD', 'RETAPPT', 'TELEPHON', 'REFERED', 'ADMITHOS', 'OTHDISP'
]

In [11]:
# Not sure to include or not
drugInformation1 = [
    'DRUGID1',
    'PRESCR1', 'CONTSUB1', 'COMSTAT1',
    'RX1CAT1', 'RX1CAT2', 'RX1CAT3', 'RX1CAT4',
    'RX1V1C1', 'RX1V1C2', 'RX1V1C3', 'RX1V1C4',
    'RX1V2C1', 'RX1V2C2', 'RX1V2C3', 'RX1V2C4',
    'RX1V3C1', 'RX1V3C2', 'RX1V3C3', 'RX1V3C4',
]
drugInformation2 = [
    'DRUGID2',
    'PRESCR2', 'CONTSUB2', 'COMSTAT2',
    'RX2CAT1', 'RX2CAT2', 'RX2CAT3', 'RX2CAT4',
    'RX2V1C1', 'RX2V1C2', 'RX2V1C3', 'RX2V1C4',
    'RX2V2C1', 'RX2V2C2', 'RX2V2C3', 'RX2V2C4',
    'RX2V3C1', 'RX2V3C2', 'RX2V3C3', 'RX2V3C4',
]
drugInformation3 = [
    'DRUGID3',
    'PRESCR3', 'CONTSUB3', 'COMSTAT3',
    'RX3CAT1', 'RX3CAT2', 'RX3CAT3', 'RX3CAT4',
    'RX3V1C1', 'RX3V1C2', 'RX3V1C3', 'RX3V1C4',
    'RX3V2C1', 'RX3V2C2', 'RX3V2C3', 'RX3V2C4',
    'RX3V3C1', 'RX3V3C2', 'RX3V3C3', 'RX3V3C4',
]
drugInformation4 = [
    'DRUGID4',
    'PRESCR4', 'CONTSUB4', 'COMSTAT4',
    'RX4CAT1', 'RX4CAT2', 'RX4CAT3', 'RX4CAT4',
    'RX4V1C1', 'RX4V1C2', 'RX4V1C3', 'RX4V1C4',
    'RX4V2C1', 'RX4V2C2', 'RX4V2C3', 'RX4V2C4',
    'RX4V3C1', 'RX4V3C2', 'RX4V3C3', 'RX4V3C4',
]
drugInformation5 = [
    'DRUGID5',
    'PRESCR5', 'CONTSUB5', 'COMSTAT5',
    'RX5CAT1', 'RX5CAT2', 'RX5CAT3', 'RX5CAT4',
    'RX5V1C1', 'RX5V1C2', 'RX5V1C3', 'RX5V1C4',
    'RX5V2C1', 'RX5V2C2', 'RX5V2C3', 'RX5V2C4',
    'RX5V3C1', 'RX5V3C2', 'RX5V3C3', 'RX5V3C4',
]
drugInformation6 = [
    'DRUGID6',
    'PRESCR6', 'CONTSUB6', 'COMSTAT6',
    'RX6CAT1', 'RX6CAT2', 'RX6CAT3', 'RX6CAT4',
    'RX6V1C1', 'RX6V1C2', 'RX6V1C3', 'RX6V1C4',
    'RX6V2C1', 'RX6V2C2', 'RX6V2C3', 'RX6V2C4',
    'RX6V3C1', 'RX6V3C2', 'RX6V3C3', 'RX6V3C4',
]
drugInformation7 = [
    'DRUGID7',
    'PRESCR7', 'CONTSUB7', 'COMSTAT7',
    'RX7CAT1', 'RX7CAT2', 'RX7CAT3', 'RX7CAT4',
    'RX7V1C1', 'RX7V1C2', 'RX7V1C3', 'RX7V1C4',
    'RX7V2C1', 'RX7V2C2', 'RX7V2C3', 'RX7V2C4',
    'RX7V3C1', 'RX7V3C2', 'RX7V3C3', 'RX7V3C4',
]
drugInformation8 = [
    'DRUGID8',
    'PRESCR8', 'CONTSUB8', 'COMSTAT8',
    'RX8CAT1', 'RX8CAT2', 'RX8CAT3', 'RX8CAT4',
    'RX8V1C1', 'RX8V1C2', 'RX8V1C3', 'RX8V1C4',
    'RX8V2C1', 'RX8V2C2', 'RX8V2C3', 'RX8V2C4',
    'RX8V3C1', 'RX8V3C2', 'RX8V3C3', 'RX8V3C4',
]
drugInformation = [
    drugInformation1, drugInformation2, drugInformation3, drugInformation4,
    drugInformation5, drugInformation6, drugInformation7, drugInformation8
]

diagnosisRecode = [
    'DIAG1R', 'DIAG2R', 'DIAG3R',
    'SCOPWI1R', 'SCOPWI2R',
    'DIAGSC1R', 'DIAGSC2R',
    'OTHPROC1R', 'OTHPROC2R', 'OTHPROC3R', 'OTHPROC4R',
]

In [12]:
# Get a list of all DataFrames in the current environment
# and clean up the environment
for obj in dir():
    if isinstance(eval(obj), pd.DataFrame) and not obj.startswith('_') and obj != 'dfs':
        del globals()[obj]

In [13]:
# Set up a dictionary of variables
variables = {
    'dateOfVisit': dateOfVisit,
    'demographics': demographics,
    'payment': payment,
    'visitReason': visitReason,
    'patientClinicHistory': patientClinicHistory,
    'vitalSigns': vitalSigns,
    'imputedFields': imputedFields,
    'physicianDiagnoses': physicianDiagnoses,
    'differentialDiagnoses': differentialDiagnoses,
    'presentSymptomsStatus': presentSymptomsStatus,
    'diagnosticScreeningServices': diagnosticScreeningServices,
    'healthEducation': healthEducation,
    'nonMedicationTreatments': nonMedicationTreatments,
    'medicationsAndImmunizations': medicationsAndImmunizations,
    'providersSeen': providersSeen,
    'visitDisposition': visitDisposition,
    #'drugInformation': drugInformation,
    #'diagnosisRecode': diagnosisRecode

}

# Check the existence of all variables among the DataFrames in `dfs`
for var_name, var in variables.items():
    # Continue if all variables are in all the DataFrames
    if all(all(v in df.columns for v in var) for df in dfs): continue

    # Print the DataFrames that do not have all the variables
    print(f'For `{var_name}` variables:\n{var}')
    for v in var:
        for df in dfs:
            if v not in df.columns:
                print(f'{v} is not in {df.file.unique()[0]}')
    print()

For `dateOfVisit` variables:
['VMONTH', 'VYEAR', 'VDAYR', 'YEAR']
VYEAR is not in opd2010.csv
VYEAR is not in opd2011.csv

For `demographics` variables:
['AGE', 'SEX', 'PREGNANT', 'GESTWEEK', 'ETHNIC', 'RACE', 'USETOBAC', 'NOTOBAC']
PREGNANT is not in opd2007.csv
PREGNANT is not in opd2008.csv
PREGNANT is not in opd2009.csv
PREGNANT is not in opd2010.csv
PREGNANT is not in opd2011.csv
GESTWEEK is not in opd2006.csv
GESTWEEK is not in opd2007.csv
GESTWEEK is not in opd2008.csv
GESTWEEK is not in opd2009.csv
GESTWEEK is not in opd2010.csv
GESTWEEK is not in opd2011.csv
ETHNIC is not in opd2007.csv
ETHNIC is not in opd2008.csv
ETHNIC is not in opd2009.csv
ETHNIC is not in opd2010.csv
ETHNIC is not in opd2011.csv
RACE is not in opd2007.csv
RACE is not in opd2008.csv
RACE is not in opd2009.csv
RACE is not in opd2010.csv
RACE is not in opd2011.csv
NOTOBAC is not in opd2007.csv
NOTOBAC is not in opd2008.csv
NOTOBAC is not in opd2009.csv
NOTOBAC is not in opd2010.csv
NOTOBAC is not in opd2011.

### 4 - Handling missing variables in the 2007-2011 dataframes

In [14]:
# Make a copy of the list of DataFrames to dfs_cleaned
dfs_cleaned = [df.copy() for df in dfs]

In [15]:
# From year 2010,
# The year of visit item (VYEAR) is no longer included on the public use file. Although the NHAMCS
# reporting periods will often begin in the last week of December and end in the last week of the
# following December, they are designed to yield statistics that are representative of the actual
# calendar year. The survey variable YEAR continues to be on the file and all visit dates may be
# assumed to reflect the calendar year. If more specific information is required, it is necessary to
# access the data through the NCHS Research Data Center.

# Impute the `VYEAR` column with the year from the `YEAR` column for the years 2010 and 2011
for df in dfs_cleaned:
    if 'VYEAR' not in df.columns:
        df['VYEAR'] = df['YEAR']

In [16]:
# From year 2007,
# Items "Is female patient pregnant, and, if so, specify gestation week." are deleted.

# Remove the `PREGNANT` and `GESTWEEK` columns from the `demographics` list
demographics.remove('PREGNANT')
demographics.remove('GESTWEEK')

In [17]:
# From year 2007,
# The high amounts of missing data are of concern both from a data collection standpoint as well as an
# analytic one. In order to highlight this problem for data users, an unimputed race variable has been added
# to each file, along with an unimputed ethnicity variable. Imputed race and ethnicity variables are included
# as usual.

# Remove `ETHNICFL` and `RACEFL` from the `imputedFields` list
imputedFields.remove('ETHNICFL')
imputedFields.remove('RACEFL')

# Replace the `RACE` and `ETHNIC` columns with the unimputed values from `RACEUN` and `ETHUN` columns from the year 2007
for df in dfs_cleaned:
    if 'RACEUN' in df.columns:
        df['RACE'] = df['RACEUN']
        df['ETHNIC'] = df['ETHUN']

In [18]:
# From year 2007,
# Tobacco use – [USETOBAC] The format of this item was modified to three checkboxes:
# not current, current and unknown. In 2006, if “Not Current” had been checked, two
# additional items, “Never and “Former” were asked. These were deleted for 2007.

# Remove the `NOTOBAC` column from the `demographics` list
demographics.remove('NOTOBAC')

In [19]:
# From year 2008,
# the variable PAYTYPE (Expected Primary Source of Payment for this Visit) has been renamed as 
# PAYTYPER (Recoded Expected Primary Source of Payment for this Visit).
# This is intended to emphasize the fact that PAYTYPER is a recoded item which uses a hierarchy
# to assign a primary expected source of payment based on the collection of multiple expected
# sources of payment.

# Replace `PAYTYPE` with `PAYTYPER` in the `payment` list
payment.remove('PAYTYPE')
payment.append('PAYTYPER')

# Rename `PAYTYPE` to `PAYTYPER` in the DataFrames for year 2006 and 2007
for df in dfs_cleaned:
    if 'PAYTYPE' in df.columns:
        df.rename(columns={'PAYTYPE': 'PAYTYPER'}, inplace=True)

In [20]:
# From year 2009,
# The Services section reflects responses to both item 7, Diagnostic/Screening Services,
# and item 9, Non-Medication Treatment.

# Combine the `diagnosticScreeningServices` and `nonMedicationTreatments` lists into a single list `services`
services = diagnosticScreeningServices + nonMedicationTreatments

# Remove `nonMedicationTreatments` from the `variables` dictionary
del variables['nonMedicationTreatments']

# Replace `diagnosticScreeningServices` with `services` in the `variables` dictionary
new_variables = {}
for var_name, var in variables.items():
    if var_name == 'diagnosticScreeningServices':
        new_variables['services'] = services
    else:
        new_variables[var_name] = var

variables = new_variables
del new_variables

In [21]:
# From year 2007,
# The previous single ultrasound checkbox was replaced with two checkboxes,
# one for echocardiogram [ECHOCARD], the other for other ultrasound [OTHULTRA].

# Add `ECHOCARD` and `OTHULTRA` to the `services` list
services.append('ECHOCARD')
services.append('OTHULTRA')

# Assign 'Yes' to the `ULTRASND` column if either `ECHOCARD` or `OTHULTRA` is 'Yes'
# Assign 'No' to the `ULTRASND` column if both `ECHOCARD` and `OTHULTRA` are 'No'
# Assign NaN to the `ULTRASND` column if both `ECHOCARD` and `OTHULTRA` are NaN
for df in dfs_cleaned:
    if 'ECHOCARD' in df.columns and 'OTHULTRA' in df.columns:
        df['ULTRASND'] = df['ECHOCARD'].combine_first(df['OTHULTRA'])

In [22]:
# `SCOPEWI1` and `SCOPEWI1` in the year 2007 and 2008 represent two Scope Procedures written-in fields.
# `DIAGSC1` and `DIAGSC2` in the year 2007 and 2008 represent two other Diagnostic Screening Procedures written-in fields.
# `OTHPROC1` to `OTHPROC4` in the year 2007 and 2008 represent four outher Surgical or Non-surgical Procedures written-in fields.
# A write-in box for site of biopsy was added to the existing checkbox for biopsy [BIOPSYWI] in the year 2007 and 2008.
# From year 2009,
# These eight fields are replaced by `PROC1` to `PROC9`,
# presenting a complete picture of the number and type of procedures reported at a visit.
# Up to 2 scope procedures, 1 biopsy site, 2 other diagnostic/screening tests/services,
# and up to 4 procedures in the non-medication treatment item could be coded
# for each outpatient department visit.

# Add columns `PROC1` to `PROC9` to the `services` list
services.extend([f'PROC{i}' for i in range(1, 10)])

# Remove `SCOPEWI1` and `SCOPEWI2` from the `service` list
services.remove('SCOPEWI1')
services.remove('SCOPEWI2')

# Remove `DIAGSC1` and `DIAGSC2` from the `service` list
services.remove('DIAGSC1')
services.remove('DIAGSC2')

# Remove `OTHPROC1` to `OTHPROC4` from the `service` list
services.remove('OTHPROC1')
services.remove('OTHPROC2')
services.remove('OTHPROC3')
services.remove('OTHPROC4')

# Renew the `services` list in the `variables` dictionary
variables['services'] = services

# Add `PROC1` to `PROC9` columns to the DataFrames for year 2007 and 2008,
# assign the values from `SCOPEWI1` and `SCOPEWI2` to the `PROC1` and `PROC2` columns
# assign the values from `BIOPSYWI` to the `PROC3` column
# assign the values from `DIAGSC1` and `DIAGSC2` to the `PROC4` and `PROC5` columns
# assign the values from `OTHPROC1` to `OTHPROC4` to the `PROC6` to `PROC9` columns
for df in dfs_cleaned:
    if 'SCOPEWI1' in df.columns and 'SCOPEWI2' in df.columns:
        df['PROC1'] = df['SCOPEWI1']
        df['PROC2'] = df['SCOPEWI2']
    else:
        df['PROC1'] = None
        df['PROC2'] = None

    if 'BIOPSYWI' in df.columns: df['PROC3'] = df['BIOPSYWI']
    else: df['PROC3'] = None

    if 'DIAGSC1' in df.columns and 'DIAGSC2' in df.columns:
        df['PROC4'] = df['DIAGSC1']
        df['PROC5'] = df['DIAGSC2']
    else:
        df['PROC4'] = None
        df['PROC5'] = None

    if 'OTHPROC1' in df.columns: df['PROC6'] = df['OTHPROC1']
    else: df['PROC6'] = None

    if 'OTHPROC2' in df.columns: df['PROC7'] = df['OTHPROC2']
    else: df['PROC7'] = None

    if 'OTHPROC3' in df.columns: df['PROC8'] = df['OTHPROC3']
    else: df['PROC8'] = None

    if 'OTHPROC4' in df.columns: df['PROC9'] = df['OTHPROC4']
    else: df['PROC9'] = None

In [23]:
# In the year 2011,
# In Diagnostic/Screening Services, the checkbox “Pap test” replaces the 2010 checkboxes
# for “Pap test – conventional”, “Pap test – liquid-based”, and “Pap test – unspecified.”

# Rename `PAP` to `PAPUNSP` in the 2011 DataFrame
for df in dfs_cleaned:
    if 'PAP' in df.columns:
        df.rename(columns={'PAP': 'PAPUNSP'}, inplace=True)

In [24]:
# From year 2009,
# In item 12, Visit Disposition, checkboxes for
# no follow-up planned, return if needed PRN, and telephone follow-up planned
# were removed.

# Remove `NOFU`, `RETPRN`, and `TELEPHON` from the `visitDisposition` list
visitDisposition.remove('NOFU')
visitDisposition.remove('RETPRN')
visitDisposition.remove('TELEPHON')

In [25]:
# From year 2009,
# In item 12, Visit Disposition, checkboxes used in 2008 for 
# “refer to emergency department” and “admit to hospital”
# were combined into a single category for 2009: Refer to ER/Admit to hospital

# Remove `REFERED` and `ADMITHOS` from the `visitDisposition` list
visitDisposition.remove('REFERED')
visitDisposition.remove('ADMITHOS')

# Add `ERADMHOS` to the `visitDisposition` list
visitDisposition.append('ERADMHOS')

# Add `ERADMHOS` column to the DataFrames for year 2006 to 2008
# Assign the values from `REFERED` and `ADMITHOS` to the `ERADMHOS` column
for df in dfs_cleaned:
    if 'REFERED' in df.columns and 'ADMITHOS' in df.columns:
        df['ERADMHOS'] = df['REFERED'].combine_first(df['ADMITHOS'])

In [26]:
# Check the existence of all variables among the DataFrames in `dfs_cleaned`
# to verify the result of the cleaning process

for var_name, var in variables.items():
    # Continue if all variables are in all the DataFrames
    if all(all(v in df.columns for v in var) for df in dfs_cleaned): continue

    # Print the DataFrames that do not have all the variables
    print(f'For `{var_name}` variables:\n{var}')
    for v in var:
        for df in dfs_cleaned:
            if v not in df.columns:
                print(f'{v} is not in {df.file.unique()[0]}')
    print()

For `presentSymptomsStatus` variables:
['ARTHRTIS', 'ASTHMA', 'CANCER', 'CASTAGE', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS', 'NOCHRON', 'TOTCHRON', 'DMP']
CASTAGE is not in opd2009.csv
CASTAGE is not in opd2011.csv
DMP is not in opd2009.csv
DMP is not in opd2010.csv
DMP is not in opd2011.csv

For `services` variables:
['BREAST', 'PELVIC', 'RECTAL', 'SKIN', 'DEPRESS', 'BONEDENS', 'MAMMO', 'MRI', 'ULTRASND', 'XRAY', 'OTHIMAGE', 'CBC', 'ELECTROL', 'GLUCOSE', 'HGBA', 'CHOLEST', 'PSA', 'OTHERBLD', 'BIOPSY', 'CHLAMYD', 'PAPCONV', 'PAPLIQ', 'PAPUNSP', 'HPVDNA', 'EKG', 'SPIRO', 'URINE', 'HTTAKE', 'WTTAKE', 'TEMPTAKE', 'BLODPRES', 'CAM', 'DME', 'HOMEHLTH', 'HOSPICE', 'PT', 'RADTHER', 'SPOCTHER', 'PSYCHOTH', 'OTHMNTL', 'EXCISION', 'ORTHO', 'WOUND', 'ECHOCARD', 'OTHULTRA', 'PROC1', 'PROC2', 'PROC3', 'PROC4', 'PROC5', 'PROC6', 'PROC7', 'PROC8', 'PROC9']
ELECTROL is not in opd2009.csv
ELECTROL is not in opd2010.csv
ELECTROL is not in opd201

In [27]:
# For year 2009 and 2011,
# “Regardless of the diagnoses written in 5a, does the patient now have:”,
# the sub-item for cancer stage was removed.

# We would want to keep the `CASTAGE` variable for the moment.

In [28]:
# From year 2009,
# “Status of patient enrollment in a disease management program
# for any of the conditions marked in 5b” was removed.

# We would want to keep the `DMP` variable for the moment.

In [29]:
# From year 2009,
# In Diagnostic /Screening Services, checkboxes for PET scan, electrolytes, and
# spirometry/pulmonary function test were removed.

# We would want to keep the `ELECTROL` and `SPIRO` variables for the moment.

In [30]:
# From year 2009,
# In Non-medication Treatment, checkboxes for
# orthopedic care, hospice care, and radiation therapy were removed.
# Radiation therapy was added back to the form in 2010

# We would want to keep the `ORTHO`, `HOSPICE`, and `RADTHER` variables for the moment.

In [31]:
# Print the update lists of variables
print('The updated list of variables is:')
print()
for var_name, var in variables.items():
    print(f'{var_name}: {var}')
    print()

print('-' * 80)

# Print the number of variables in total
print('The number of variables in total is:')
print(sum(len(var) for var in variables.values()))

The updated list of variables is:

dateOfVisit: ['VMONTH', 'VYEAR', 'VDAYR', 'YEAR']

demographics: ['AGE', 'SEX', 'ETHNIC', 'RACE', 'USETOBAC']

payment: ['PAYPRIV', 'PAYMCARE', 'PAYMCAID', 'PAYWKCMP', 'PAYSELF', 'PAYNOCHG', 'PAYOTH', 'PAYDK', 'PAYTYPER']

visitReason: ['INJDET', 'INJURY', 'MAJOR', 'RFV1', 'RFV2', 'RFV3']

patientClinicHistory: ['SENBEFOR', 'PASTVIS']

vitalSigns: ['HTIN', 'WTLB', 'BMI', 'TEMPF', 'BPSYS', 'BPDIAS']

imputedFields: ['BDATEFL', 'SEXFL', 'SENBEFL', 'PASTFL']

physicianDiagnoses: ['DIAG1', 'DIAG2', 'DIAG3']

differentialDiagnoses: ['PRDIAG1', 'PRDIAG2', 'PRDIAG3']

presentSymptomsStatus: ['ARTHRTIS', 'ASTHMA', 'CANCER', 'CASTAGE', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS', 'NOCHRON', 'TOTCHRON', 'DMP']

services: ['BREAST', 'PELVIC', 'RECTAL', 'SKIN', 'DEPRESS', 'BONEDENS', 'MAMMO', 'MRI', 'ULTRASND', 'XRAY', 'OTHIMAGE', 'CBC', 'ELECTROL', 'GLUCOSE', 'HGBA', 'CHOLEST', 'PSA', 'OTHERBLD', 'BIOPSY', 

### 5 - Add new variables in the 2007-2011 dataframes to the lists of variables
The initial lists of variables are created based on the 2006 data and document.

There are some new fields added to the survey and data since 2007.

In [32]:
# From year 2007,

# In Diagnostic/Screening Services – New checkboxes were added for
# CT scan [CATSCAN], PET scan [PETSCAN], and pregnancy test [PREGTEST]

# PET scan [PETSCAN] is removed again from year 2009 to 2011, So we don't add it to the `services` list.

# Add `CATSCAN`, `PETSCAN`, `PREGTEST`, and `BIOPSYWI` to the `services` list
services.append('CATSCAN')
#services.append('PETSCAN')
services.append('PREGTEST')


# In Providers – A checkbox was added for mental health provider [MHP].

# Add `MHP` to the `providersSeen` list
providersSeen.append('MHP')

In [33]:
# From year 2009,

# In item 7, Diagnostic /Screening Services, there are new checkboxes for:
# Foot examination, Retinal examination, HIV test

# Add `FOOT`, `RETINAL`, `HIVTEST` to the `services` list
services.append('FOOT')
services.append('RETINAL')
services.append('HIVTEST')


# In item 9, Non-medication Treatment, there are new checkboxes for:
# Cast, Splint or wrap

# Add `CAST`, `SPLINT` to the `services` list
services.append('CAST')
services.append('SPLINT')

In [34]:
# Check the existence of all variables among the DataFrames in `dfs_cleaned`
# to verify the result of the cleaning process

for var_name, var in variables.items():
    # Continue if all variables are in all the DataFrames
    if all(all(v in df.columns for v in var) for df in dfs_cleaned): continue

    # Print the DataFrames that do not have all the variables
    print(f'For `{var_name}` variables:\n{var}')
    for v in var:
        for df in dfs_cleaned:
            if v not in df.columns:
                print(f'{v} is not in {df.file.unique()[0]}')
    print()

For `presentSymptomsStatus` variables:
['ARTHRTIS', 'ASTHMA', 'CANCER', 'CASTAGE', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS', 'NOCHRON', 'TOTCHRON', 'DMP']
CASTAGE is not in opd2009.csv
CASTAGE is not in opd2011.csv
DMP is not in opd2009.csv
DMP is not in opd2010.csv
DMP is not in opd2011.csv

For `services` variables:
['BREAST', 'PELVIC', 'RECTAL', 'SKIN', 'DEPRESS', 'BONEDENS', 'MAMMO', 'MRI', 'ULTRASND', 'XRAY', 'OTHIMAGE', 'CBC', 'ELECTROL', 'GLUCOSE', 'HGBA', 'CHOLEST', 'PSA', 'OTHERBLD', 'BIOPSY', 'CHLAMYD', 'PAPCONV', 'PAPLIQ', 'PAPUNSP', 'HPVDNA', 'EKG', 'SPIRO', 'URINE', 'HTTAKE', 'WTTAKE', 'TEMPTAKE', 'BLODPRES', 'CAM', 'DME', 'HOMEHLTH', 'HOSPICE', 'PT', 'RADTHER', 'SPOCTHER', 'PSYCHOTH', 'OTHMNTL', 'EXCISION', 'ORTHO', 'WOUND', 'ECHOCARD', 'OTHULTRA', 'PROC1', 'PROC2', 'PROC3', 'PROC4', 'PROC5', 'PROC6', 'PROC7', 'PROC8', 'PROC9', 'CATSCAN', 'PREGTEST', 'FOOT', 'RETINAL', 'HIVTEST', 'CAST', 'SPLINT']
ELECTROL is not 

In [35]:
# Print the update lists of variables
print('The updated list of variables is:')
print()
for var_name, var in variables.items():
    print(f'{var_name}: {var}')
    print()

print('-' * 80)

# Print the number of variables in total
print('The number of variables in total is:')
print(sum(len(var) for var in variables.values()))

The updated list of variables is:

dateOfVisit: ['VMONTH', 'VYEAR', 'VDAYR', 'YEAR']

demographics: ['AGE', 'SEX', 'ETHNIC', 'RACE', 'USETOBAC']

payment: ['PAYPRIV', 'PAYMCARE', 'PAYMCAID', 'PAYWKCMP', 'PAYSELF', 'PAYNOCHG', 'PAYOTH', 'PAYDK', 'PAYTYPER']

visitReason: ['INJDET', 'INJURY', 'MAJOR', 'RFV1', 'RFV2', 'RFV3']

patientClinicHistory: ['SENBEFOR', 'PASTVIS']

vitalSigns: ['HTIN', 'WTLB', 'BMI', 'TEMPF', 'BPSYS', 'BPDIAS']

imputedFields: ['BDATEFL', 'SEXFL', 'SENBEFL', 'PASTFL']

physicianDiagnoses: ['DIAG1', 'DIAG2', 'DIAG3']

differentialDiagnoses: ['PRDIAG1', 'PRDIAG2', 'PRDIAG3']

presentSymptomsStatus: ['ARTHRTIS', 'ASTHMA', 'CANCER', 'CASTAGE', 'CEBVD', 'CHF', 'CRF', 'COPD', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS', 'NOCHRON', 'TOTCHRON', 'DMP']

services: ['BREAST', 'PELVIC', 'RECTAL', 'SKIN', 'DEPRESS', 'BONEDENS', 'MAMMO', 'MRI', 'ULTRASND', 'XRAY', 'OTHIMAGE', 'CBC', 'ELECTROL', 'GLUCOSE', 'HGBA', 'CHOLEST', 'PSA', 'OTHERBLD', 'BIOPSY', 

### 6 - Merge the dataframes with the selected variables of interest and clean up the values

In [36]:
# Concatenate all selected variables of interest into a list
selected_vars = list()
for var in variables.values():
    selected_vars.extend(var)

# Add in the `file` column to the list of selected variables
selected_vars = ['file'] + selected_vars

# Merge the DataFrames with the selected variables of interest
df = pd.concat(dfs_cleaned, ignore_index=True)
df = df[selected_vars]

# Check the shape of the merged DataFrame
print('The shape of the merged DataFrame is:')
print(df.shape)

The shape of the merged DataFrame is:
(203988, 161)


In [37]:
# Check the unique values of each column in the merged DataFrame,
# and find those columns that the unique values include both numeric and non-numeric values

# Get the unique values of each column
unique_values = df.apply(lambda x: x.dropna().unique())

# Go through the unique values of each column,
# and find those columns that contains both numeric and non-numeric values using Regular Expression
mixed_value_cols = []
for col, values in unique_values.items():
    #if any(re.search(r'[\d.]', str(v)) for v in values) and any(re.search(r'[^\d.]', str(v)) for v in values):
    if any(re.fullmatch(r'\d*\.?\d+', str(v)) for v in values) and any(not re.fullmatch(r'\d*\.?\d+', str(v)) for v in values):
        mixed_value_cols.append(col)

# Print the columns that contain both numeric and non-numeric values
print('The columns that contain both numeric and non-numeric values are:')
print(mixed_value_cols)

The columns that contain both numeric and non-numeric values are:
['AGE', 'PASTVIS', 'HTIN', 'WTLB', 'BMI', 'TEMPF', 'BPSYS', 'BPDIAS', 'DIAG1', 'DIAG2', 'DIAG3', 'TOTCHRON', 'PROC6', 'PROC7', 'MED1', 'MED2', 'MED3', 'MED4', 'MED5']


In [38]:
# Check the values of those including non-numeric values in the `AGE` column
print('The values of those including non-numeric values in the `AGE` column are:')
print(df.loc[df['BMI'].astype(str).str.contains(r'[^\d.]'), 'AGE'].unique())

# !!! Replace 'Under one year' with 1 in the `AGE` column
df['AGE'] = df['AGE'].replace('Under one year', 1)

# Replace '91 years or older' with 91 in the `AGE` column
df['AGE'] = df['AGE'].replace('91 years or older', 91)

# Replace '100 years and over' with 100 in the `AGE` column
df['AGE'] = df['AGE'].replace('100 years and over', 100)

# Cast the `AGE` column to float
df['AGE'] = df['AGE'].astype(float)

The values of those including non-numeric values in the `AGE` column are:
['37.0' '48.0' '40.0' '46.0' '34.0' '83.0' '45.0' '65.0' '26.0' '39.0'
 '80.0' '16.0' '81.0' '57.0' '30.0' '61.0' '58.0' '50.0' '91.0' '20.0'
 '56.0' '59.0' '28.0' '35.0' '66.0' '69.0' '64.0' '51.0' '25.0' '29.0'
 '23.0' '27.0' '22.0' '41.0' '31.0' '36.0' '42.0' '32.0' '24.0' '33.0'
 '38.0' '44.0' '21.0' '47.0' '19.0' '76.0' '18.0' '49.0' '43.0' '52.0'
 '55.0' '74.0' '60.0' '54.0' '63.0' '67.0' '85.0' '17.0' '72.0' '62.0'
 '70.0' '87.0' '86.0' '75.0' '84.0' '68.0' '71.0' '77.0' '79.0' '78.0'
 '6.0' '9.0' 'Under one year' '10.0' '1.0' '5.0' '11.0' '53.0' '73.0'
 '8.0' '3.0' '2.0' '7.0' '4.0' '15.0' '89.0' '14.0' '82.0' '12.0' '88.0'
 '90.0' '92.0' '96.0' '13.0' '100 years and over' '94.0' '93.0' '95.0'
 '97.0' '98.0' '99.0' '91 years or older']


In [39]:
# Check the values of those including non-numeric values in the `PASTVIS` column
print('The values of those including non-numeric values in the `PASTVIS` column are:')
print(df.loc[df['PASTVIS'].astype(str).str.contains(r'[^\d.]'), 'PASTVIS'].unique())

# Check the value counts of `SENBEFOR` for the rows where `PASTVIS` is 'Not applicable'
print()
print('The value counts of `SENBEFOR` for the rows where `PASTVIS` is `Not applicable` are:')
print(df.loc[df['PASTVIS'] == 'Not applicable', 'SENBEFOR'].value_counts())


# ITEM 4b. HAS THE PATIENT BEEN SEEN IN THIS CLINIC BEFORE?
# “Seen” means “provided care for” at any time in the past. Mark “Yes, established patient” if the patient
# was seen before by any physician or staff member in the clinic. Exclude this visit.
# Mark “No, new patient” if the patient has not been seen in the clinic before.
# If “Yes” is checked, also indicate approximately how many past visits the patient has made to this clinic
# within the last 12 months using the check boxes provided. Do not include the current visit in your total. If
# you cannot determine how many past visits were made, then please mark “Unknown.” Include all visits to
# other physicians or health care providers in this clinic.

# Replace the value 'Not applicable' with 0 in the `PASTVIS` column,
# since 'Not applicable' means, 'No, new patient' has been checked in the `SENBEFOR` column.
df['PASTVIS'] = df['PASTVIS'].replace('Not applicable', 0)

# Check the values of those including non-numeric values in the `PASTVIS` column
print()
print('The values of those including non-numeric values in the `PASTVIS` column are:')
print(df.loc[df['PASTVIS'].astype(str).str.contains(r'[^\d.]'), 'PASTVIS'].unique())

# ! Replace the value '99 visits or more' with 99 in the `PASTVIS` column
df['PASTVIS'] = df['PASTVIS'].replace('99 visits or more', 99)

# ! Replace the value '75 visits or more (CLINTYPE 2,6)' with 75 in the `PASTVIS` column
df['PASTVIS'] = df['PASTVIS'].replace('75 visits or more (CLINTYPE 2,6)', 75)

# ! Replace the value '34 visits or more' and '34 visits or more (CLINTYPE 1,3,4)' with 34 in the `PASTVIS` column
df['PASTVIS'] = df['PASTVIS'].replace('34 visits or more', 34)
df['PASTVIS'] = df['PASTVIS'].replace('34 visits or more (CLINTYPE 1,3,4)', 34)

# !!! Replace the value '6 visits or more' and '6 or more' with 6 in the `PASTVIS` column
df['PASTVIS'] = df['PASTVIS'].replace('6 visits or more', 6)
df['PASTVIS'] = df['PASTVIS'].replace('6 or more', 6)

# !!! Replace the value '1-2' with 1.5 in the `PASTVIS` column
df['PASTVIS'] = df['PASTVIS'].replace('1-2', 1.5)

# !!! Replace the value '3-5' with 4 in the `PASTVIS` column
df['PASTVIS'] = df['PASTVIS'].replace('3-5', 4)

# Cast the `PASTVIS` column to float
df['PASTVIS'] = df['PASTVIS'].astype(float)

The values of those including non-numeric values in the `PASTVIS` column are:
['3-5' '1-2' '6 or more' 'Not applicable' nan '99 visits or more'
 '75 visits or more (CLINTYPE 2,6)' '34 visits or more (CLINTYPE 1,3,4)']

The value counts of `SENBEFOR` for the rows where `PASTVIS` is `Not applicable` are:
SENBEFOR
No, new patient    39270
Name: count, dtype: int64

The values of those including non-numeric values in the `PASTVIS` column are:
['3-5' '1-2' '6 or more' nan '99 visits or more'
 '75 visits or more (CLINTYPE 2,6)' '34 visits or more (CLINTYPE 1,3,4)']


In [40]:
# Check the values of those including non-numeric values in the `HTIN` column
print('The values of those including non-numeric values in the `HTIN` column are:')
print(df.loc[df['HTIN'].astype(str).str.contains(r'[^\d.]'), 'HTIN'].unique())

# Replace 'Blank' with Nan in the `HTIN` column
df['HTIN'] = df['HTIN'].replace('Blank', float('nan'))

# !!! Replace '72 inches or more (females)' with 72 in the `HTIN` column
df['HTIN'] = df['HTIN'].replace('72 inches or more (females)', 72)

# !!! Replace '77 inches or more (males)' with 77 in the `HTIN` column
df['HTIN'] = df['HTIN'].replace('77 inches or more (males)', 77)

# Cast the `HTIN` column to float
df['HTIN'] = df['HTIN'].astype(float)

The values of those including non-numeric values in the `HTIN` column are:
['Blank' '72 inches or more (females)' '77 inches or more (males)']


In [41]:
# Check the values of those including non-numeric values in the `WTLB` column
print('The values of those including non-numeric values in the `WTLB` column are:')
print(df.loc[df['WTLB'].astype(str).str.contains(r'[^\d.]'), 'WTLB'].unique())

# Replace 'Blank' with Nan in the `WTLB` column
df['WTLB'] = df['WTLB'].replace('Blank', float('nan'))

# !!! Replace '500 lbs. or more' with 500 in the `WTLB` column
df['WTLB'] = df['WTLB'].replace('500 lbs. or more', 500)

# !!! Replace '385 lbs. or more' with 385 in the `WTLB` column
df['WTLB'] = df['WTLB'].replace('385 lbs. or more', 385)

# Cast the `WTLB` column to float
df['WTLB'] = df['WTLB'].astype(float)

The values of those including non-numeric values in the `WTLB` column are:
['Blank' '500 lbs. or more' '385 lbs. or more']


In [42]:
# Check the values of those including non-numeric values in the `BMI` column
print('The values of those including non-numeric values in the `BMI` column are:')
print(df.loc[df['BMI'].astype(str).str.contains(r'[^\d.]'), 'BMI'].unique())

# Replace non-numeric values in the `BMI` column with NaN
df['BMI'] = pd.to_numeric(df['BMI'], errors='coerce')   # invalid parsing will be set as NaN with errors='coerce'

# Replace the value 999.00 with NaN in the `BMI` column
df['BMI'] = df['BMI'].replace(999.00, float('nan'))

# Cast the `BMI` column to float
df['BMI'] = df['BMI'].astype(float)

The values of those including non-numeric values in the `BMI` column are:
['Blank' 'Not applicable' 'Not calculated' 'Missing data']


In [43]:
# Check the values of those including non-numeric values in the `TEMPF` column
print('The values of those including non-numeric values in the `TEMPF` column are:')
print(df.loc[df['TEMPF'].astype(str).str.contains(r'[^\d.]'), 'TEMPF'].unique())

# Replace 'Blank' with NaN in the `TEMPF` column
df['TEMPF'] = df['TEMPF'].replace('Blank', float('nan'))

# Cast the `TEMPF` column to float
df['TEMPF'] = df['TEMPF'].astype(float)

The values of those including non-numeric values in the `TEMPF` column are:
['Blank']


In [44]:
# Check the values of those including non-numeric values in the `BPSYS` column
print('The values of those including non-numeric values in the `BPSYS` column are:')
print(df.loc[df['BPSYS'].astype(str).str.contains(r'[^\d.]'), 'BPSYS'].unique())

# Replace 'Blank' with NaN in the `BPSYS` column
df['BPSYS'] = df['BPSYS'].replace('Blank', float('nan'))

# Cast the `BPSYS` column to float
df['BPSYS'] = df['BPSYS'].astype(float)

The values of those including non-numeric values in the `BPSYS` column are:
['Blank']


In [45]:
# Check the values of those including non-numeric values in the `BPDIAS` column
print('The values of those including non-numeric values in the `BPDIAS` column are:')
print(df.loc[df['BPDIAS'].astype(str).str.contains(r'[^\d.]'), 'BPDIAS'].unique())

# Replace 'Blank' with NaN in the `BPDIAS` column
df['BPDIAS'] = df['BPDIAS'].replace('Blank', float('nan'))

# !!! Replace 'P, PALP, DOPP or DOPPLER' with NaN in the `BPDIAS` column
df['BPDIAS'] = df['BPDIAS'].replace('P, PALP, DOPP or DOPPLER', float('nan'))

# Cast the `BPDIAS` column to float
df['BPDIAS'] = df['BPDIAS'].astype(float)

The values of those including non-numeric values in the `BPDIAS` column are:
['Blank' 'P, PALP, DOPP or DOPPLER']


In [46]:
# Check the values of those including non-numeric values in the `TOTCHRON` column
print('The values of those including non-numeric values in the `TOTCHRON` column are:')
print(df.loc[df['TOTCHRON'].astype(str).str.contains(r'[^\d.]'), 'TOTCHRON'].unique())

# Replace 'Entire item blank' with NaN in the `TOTCHRON` column
df['TOTCHRON'] = df['TOTCHRON'].replace('Entire item blank', float('nan'))

# Cast the `TOTCHRON` column to float
df['TOTCHRON'] = df['TOTCHRON'].astype(float)

The values of those including non-numeric values in the `TOTCHRON` column are:
['Entire item blank']


In [47]:
# Check the value counts of `CASTAGE`
print('The value counts of `CASTAGE` are:')
print(df['CASTAGE'].value_counts())

# Replace the value 'Not applicable', 'Unknown', 'Unknown stage', and 'No box is marked' with None in the `CASTAGE` column
df['CASTAGE'] = df['CASTAGE'].replace('Not applicable', None)
df['CASTAGE'] = df['CASTAGE'].replace('Unknown', None)
df['CASTAGE'] = df['CASTAGE'].replace('Unknown stage', None)
df['CASTAGE'] = df['CASTAGE'].replace('No box is marked', None)

# Check the value counts of `CASTAGE`
print()
print('The value counts of `CASTAGE` are:')
print(df['CASTAGE'].value_counts())

The value counts of `CASTAGE` are:
CASTAGE
Not applicable      129162
Unknown               3274
Unknown stage         1626
Local                 1392
In situ                693
Distant                668
Regional               625
Stage IV               247
Stage III              151
Stage II               140
Stage 1                128
No box is marked        98
Name: count, dtype: int64

The value counts of `CASTAGE` are:
CASTAGE
Local        1392
In situ       693
Distant       668
Regional      625
Stage IV      247
Stage III     151
Stage II      140
Stage 1       128
Name: count, dtype: int64


In [48]:
# Clean up the other columns which may have 'Unknown' or 'Blank' values

# Check the columns that contain 'Unknown' or 'Blank' values
unknown_blank_cols = df.columns[df.isin(['Unknown', 'Blank']).any()]

# Print the columns that contain 'Unknown' or 'Blank' values
print('The columns that contain `Unknown` or `Blank` values are:')
print(unknown_blank_cols)

# Replace 'Unknown' and 'Blank' with None in the columns that contain 'Unknown' or 'Blank' values
df[unknown_blank_cols] = df[unknown_blank_cols].replace(['Unknown', 'Blank'], None)

The columns that contain `Unknown` or `Blank` values are:
Index(['ETHNIC', 'RACE', 'USETOBAC', 'PAYTYPER', 'INJDET', 'MAJOR', 'RFV1',
       'RFV2', 'RFV3', 'DIAG1', 'DIAG2', 'DIAG3', 'DMP', 'PROC1', 'PROC2',
       'PROC3', 'PROC4', 'PROC5', 'PROC6', 'PROC7', 'PROC8', 'PROC9', 'NCMED1',
       'NCMED2', 'NCMED3', 'NCMED4', 'NCMED5', 'NCMED6', 'NCMED7', 'NCMED8'],
      dtype='object')


### 7 - Split and save the cleaned OPD DataFrame to a CSV file in '..data/cleaned/'

In [49]:
# Hold aside the 2011 data for final evaluation on the model
# of the generalizability and adaptability to potential shifts in data over time
df_2011 = df[df['file'].str.contains('2011')]

# Set the 2010 data as the test set
test_df = df[df['file'].str.contains('2010')]

# Set the 2009 data as the validation set
val_df = df[df['file'].str.contains('2009')]

# Set the 2006, 2007, and 2008 data as the training set
train_df = df[df['file'].str.contains('2006') | df['file'].str.contains('2007') | df['file'].str.contains('2008')]

# Check the number of rows and proportion of the data in each set
print('The number of rows and proportion of the data in each set are:')
print()
print(f'Training Data (2006-2008): {train_df.shape[0]} rows, approximately {train_df.shape[0] / df.shape[0]:.1%} of the total')
print(f'Validation Data (2009): {val_df.shape[0]} rows, approximately {val_df.shape[0] / df.shape[0]:.1%} of the total')
print(f'Test Data (2010): {test_df.shape[0]} rows, approximately {test_df.shape[0] / df.shape[0]:.1%} of the total')
print()
print(f'Final Evaluation Data (2011): {df_2011.shape[0]} rows, approximately {df_2011.shape[0] / df.shape[0]:.1%} of the total')

The number of rows and proportion of the data in each set are:

Training Data (2006-2008): 103486 rows, approximately 50.7% of the total
Validation Data (2009): 33551 rows, approximately 16.4% of the total
Test Data (2010): 34718 rows, approximately 17.0% of the total

Final Evaluation Data (2011): 32233 rows, approximately 15.8% of the total


In [50]:
# Export the cleaned data to CSV files
save_path = os.path.join('..', 'data', 'cleaned')
if not os.path.exists(save_path):
    os.makedirs(save_path)

train_df.to_csv(os.path.join(save_path, 'train.csv'), index=False)
val_df.to_csv(os.path.join(save_path, 'val.csv'), index=False)
test_df.to_csv(os.path.join(save_path, 'test.csv'), index=False)
df_2011.to_csv(os.path.join(save_path, 'final_evaluation.csv'), index=False)